In [1]:
# get text and format it    


with open('Appel avec Nicolas CASTET-20241030_103642-Transcription de la réunion-txt.txt', encoding="latin-1") as f:
    transcript = f.readlines()
    transcript

In [3]:
# format in json to feed azure api

import re
import json



def parse_vtt(transcript):
    entries = []
    segments = re.split(r"\d+:\d+:\d+\.\d+ --> \d+:\d+:\d+\.\d+\n", transcript.strip())
    for idx, segment in enumerate(segments[1:], start=1):  # Skip the first split, which is empty
        lines = segment.strip().split('\n')
        if len(lines) >= 2:
            speaker, text = lines[0], lines[1]
            role = "Customer" if "Jaubert" in speaker else "Agent"
            entries.append({
                "text": text,
                "id": str(idx),
                "role": role,
                "participantId": speaker.replace(" ", "_"),
            })
    return entries

conversation_items = parse_vtt("".join(transcript))
conversation_json = {
    "conversations": [
        {
            "conversationItems": conversation_items,
            "modality": "text",
            "id": "conversation1",
            "language": "fr",
        }
    ]
}

print(json.dumps(conversation_json, indent=2, ensure_ascii=False))


{
  "conversations": [
    {
      "conversationItems": [
        {
          "text": "OK alors normalement c'est bon.",
          "id": "1",
          "role": "Customer",
          "participantId": "Elodie_Jaubert"
        },
        {
          "text": "La transcription, c'est à dire que c'est l'enregistrement.",
          "id": "2",
          "role": "Agent",
          "participantId": "Nicolas_CASTET"
        },
        {
          "text": "Oui, enfin l'enregistrement.",
          "id": "3",
          "role": "Customer",
          "participantId": "Elodie_Jaubert"
        },
        {
          "text": "Écrit et pas vidéo ?",
          "id": "4",
          "role": "Customer",
          "participantId": "Elodie_Jaubert"
        },
        {
          "text": "Ah oui.",
          "id": "5",
          "role": "Agent",
          "participantId": "Nicolas_CASTET"
        },
        {
          "text": "Donc il faut bien articuler pour qu'il comprenne.",
          "id": "6",
          "r

In [11]:
conversation_json['conversations']

[{'conversationItems': [{'text': "OK alors normalement c'est bon.",
    'id': '1',
    'role': 'Customer',
    'participantId': 'Elodie_Jaubert'},
   {'text': "La transcription, c'est à dire que c'est l'enregistrement.",
    'id': '2',
    'role': 'Agent',
    'participantId': 'Nicolas_CASTET'},
   {'text': "Oui, enfin l'enregistrement.",
    'id': '3',
    'role': 'Customer',
    'participantId': 'Elodie_Jaubert'},
   {'text': 'Écrit et pas vidéo\xa0?',
    'id': '4',
    'role': 'Customer',
    'participantId': 'Elodie_Jaubert'},
   {'text': 'Ah oui.',
    'id': '5',
    'role': 'Agent',
    'participantId': 'Nicolas_CASTET'},
   {'text': "Donc il faut bien articuler pour qu'il comprenne.",
    'id': '6',
    'role': 'Agent',
    'participantId': 'Nicolas_CASTET'},
   {'text': 'Mais franchement ça va, hein\xa0?',
    'id': '7',
    'role': 'Customer',
    'participantId': 'Elodie_Jaubert'},
   {'text': 'Il comprend super bien et il écrit super vite.',
    'id': '8',
    'role': 'Cust

In [12]:
import os

key = 
endpoint = 

from azure.core.credentials import AzureKeyCredential
from azure.ai.language.conversations import ConversationAnalysisClient

client = ConversationAnalysisClient(endpoint, AzureKeyCredential(key))
with client:
    poller = client.begin_conversation_analysis(
        task={
            "displayName": "Analyze conversations from xxx",
            "analysisInput": {
                "conversations": conversation_json['conversations']
            },
            "tasks": [
                {
                    "taskName": "Issue task",
                    "kind": "ConversationalSummarizationTask",
                    "parameters": {"summaryAspects": ["issue"]},
                },
                {
                    "taskName": "Resolution task",
                    "kind": "ConversationalSummarizationTask",
                    "parameters": {"summaryAspects": ["resolution"]},
                },
            ],
        }
    )

    # view result
    result = poller.result()
    task_results = result["tasks"]["items"]
    for task in task_results:
        print(f"\n{task['taskName']} status: {task['status']}")
        task_result = task["results"]
        if task_result["errors"]:
            print("... errors occurred ...")
            for error in task_result["errors"]:
                print(error)
        else:
            conversation_result = task_result["conversations"][0]
            if conversation_result["warnings"]:
                print("... view warnings ...")
                for warning in conversation_result["warnings"]:
                    print(warning)
            else:
                summaries = conversation_result["summaries"]
                for summary in summaries:
                    print(f"{summary['aspect']}: {summary['text']}")


Issue task status: succeeded
issue: Le client a des problèmes pour enregistrer la vidéo d'une réunion d'équipe. Il a pu commencer la transcription, mais pas la vidéo.

Resolution task status: succeeded
resolution: L'agent a confirmé que la transcription a été enregistrée en direct et qu'il peut être afficher. L'agent n'a pas encore fourni de solution ou de diagnostic. Le client a mentionné qu'il a pu enregistrer la vidéo d'une réunion, mais pas la transcription. L'agent et le client ont discuté de la saveur de la transcription, qui est supprimée avec le logiciel de leur ami Remi.
